### IBM Data Science Capstone Project - Part I

#### Wiki Scraping - Segmenting and Clustering Toronto

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

#### Set the URL you want to webscrape from

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' ### given url

In [3]:
response = requests.get(url)
response #200 means it went through

<Response [200]>

#### Parse html and save to BeautifulSoup object

In [4]:
soup = BeautifulSoup(response.text, "html.parser") # but let's use the pandas method instead

#### Alternative Pandas method to read html

In [5]:
from pandas.io.html import read_html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # same url as above

In [6]:
Toronto = pd.read_html(url, header = 0)
postal_data = Toronto[0]


In [7]:
Toronto[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
type(postal_data)

pandas.core.frame.DataFrame

### Prepare the dataframe 

#### remove cells with a Borough that is "Not assigned" - use a filter to locate the row and drop it

In [9]:
filt = (postal_data['Borough'] == 'Not assigned')
postal_data.drop(index = postal_data[filt].index, inplace = True)

#### sort the dataframe by Postal Code in ascending order

In [10]:
postal_data.sort_values(by = 'Postal Code', ascending = True, inplace = True)

In [11]:
postal_data

,Postal Code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [12]:
# reset the index
postal_data = postal_data.reset_index(drop=True)

In [13]:
postal_data

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


#### Check for duplicates in Postal Code column

In [14]:
# check for duplicates in Postal Code column
duplicates = postal_data[postal_data.duplicated(['Postal Code'])]
noduplicates = duplicates.empty
# print(noduplicates)

In [15]:
if noduplicates == True :
    print('No Postal Code duplicates')
else :
    print("Duplicate Postal Codes: ", duplicates, sep='\n')
    # execute code to combine two neighborhoods into one row with the neighborhood names separated with a comma
    # use GROUPBY and AGG to group duplicate postal codes and combine the respective Neighborhood names separated by comma
    postal_data.groupby('Postal Code')['Neighborhood'].agg(','.join)

No Postal Code duplicates


#### If Neighborhood is "Not assigned" then it will be the same as the Borough

In [16]:
filt2 = (postal_data['Neighborhood'] == 'Not assigned')
postal_data.loc[filt2, 'Neighborhood'] = postal_data['Borough']

### Get the postal code geo coordinates

In [17]:
tnt_coord = pd.read_csv('http://cocl.us/Geospatial_data') 
tnt_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# check the size of the table
tnt_coord.shape

(103, 3)

In [19]:
#  rename column to postal_code to avoid confusion between the two dataframes
tnt_coord.columns = ['postal_code','Latitude','Longitude']

In [20]:
tnt_coord

,postal_code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [21]:
# ensure dataframe is sorted by Postal Code in ascending order
tnt_coord.sort_values(by = 'postal_code', ascending = True, inplace = True)

In [22]:
Toronto_df = pd.concat((postal_data,tnt_coord), axis = 1)

In [23]:
Toronto_df

,Postal Code,Borough,Neighborhood,postal_code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437


In [24]:
# compare the postal codes from both columns 'Postal Code' and 'postal_code' to ensure consistency 
Toronto_df['Postal Code'].equals(Toronto_df['postal_code'])

True

In [25]:
if  True :
    print('Postal code perfect match')
else :
    print('Problem with mismatch postal code')
    

Postal code perfect match


In [26]:
# drop the column 'postal_code' from the combined dataframe
Toronto_df.drop(columns =['postal_code'], inplace = True)

#### Review final format 

In [27]:
Toronto_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### "The End" and last cell

In [28]:
Toronto_df.shape

(103, 5)